## Loading a Linear Learner Model with Apache MXNet in Python

- **Kernel**: `conda_mxnet_p36`
- **SageMaker Python SDK Version**: `2.X`

In [1]:
%store -r model_data

In [2]:
model_data

's3://sagemaker-cookbook-bucket/chapter01/output/linear-learner-2021-03-13-02-23-18-930/output/model.tar.gz'

In [3]:
import sagemaker 
session = sagemaker.Session()

In [4]:
from sagemaker.s3 import S3Downloader

In [6]:
S3Downloader.download(s3_uri=model_data,
                      local_path="tmp/",
                      sagemaker_session=session)

In [7]:
!ls tmp

model.tar.gz  training_data.csv


In [1]:
!tar -xzvf tmp/model.tar.gz

model_algo-1


In [33]:
!unzip model_algo-1 

256

In [31]:
import mxnet

from mxnet import gluon
from json import load as json_load 
from json import dumps as json_dumps

In [35]:
sym_json = json_load(open('mx-mod-symbol.json')) 
sym_json_string = json_dumps(sym_json)

model = gluon.nn.SymbolBlock( 
    outputs=mxnet.sym.load_json(sym_json_string), 
    inputs=mxnet.sym.var('data'))

model.load_parameters('mx-mod-0000.params', allow_missing=True)

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/gluon/block.py:1454: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]


In [36]:
model.initialize()

def mxnet_predict(x, model=model):
    return model(mxnet.nd.array([x]))[0].asscalar()

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/gluon/parameter.py:887: UserWarning: Parameter 'fc0_weight' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/gluon/parameter.py:887: UserWarning: Parameter 'fc0_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)


In [37]:
mxnet_predict(42)

1226.6005

In [38]:
def extract_weight_and_bias(model):
    params = model.collect_params()
    weight = params['fc0_weight'].data()[0].asscalar() 
    bias = params['fc0_bias'].data()[0].asscalar()
    
    return {
        "weight": weight, 
        "bias": bias
    } 

In [40]:
weight_and_bias = extract_weight_and_bias(model)
weight_and_bias

{'weight': 8.219234, 'bias': 881.3926}

In [41]:
%store weight_and_bias

Stored 'weight_and_bias' (dict)


In [44]:
%%bash

rm -f additional-params.json
rm -f manifest.json
rm -f model_algo-1
rm -f mx-mod-symbol.json
rm -f mx-mod-0000.params